In [1]:
import pandas as pd
import numpy as np

In [2]:
#!pip install cvxpy

In [3]:
#import cvxpy as cp

In [4]:
#First, need a function that picks elements from list L with probabilies probs
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

# Read in Data

### Option 1: Read in DR projections and DK salaries, and make sure names match

In [15]:
df = pd.read_csv('https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=129&draftGroupId=32040')
df.iloc[np.r_[0:3,-3:0]]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Justin Thomas (13919919),Justin Thomas,13919919,CPT,16200,Presidents Cup,Golf,102.70
1,G,Xander Schauffele (13919920),Xander Schauffele,13919920,CPT,15300,Presidents Cup,Golf,90.25
2,G,Adam Scott (13919921),Adam Scott,13919921,CPT,14400,Presidents Cup,Golf,86.62
45,G,Hao-Tong Li (13919916),Hao-Tong Li,13919916,G,4400,Presidents Cup,Golf,71.93
46,G,Adam Hadwin (13919917),Adam Hadwin,13919917,G,4200,Presidents Cup,Golf,83.60
47,G,C.T. Pan (13919918),C.T. Pan,13919918,G,4000,Presidents Cup,Golf,51.43


Account for R1

In [16]:
df[df.Name.str.contains('Smith')]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
18,G,Cameron Smith (13919932),Cameron Smith,13919932,CPT,9000,Presidents Cup,Golf,88.33
36,G,Cameron Smith (13919908),Cameron Smith,13919908,G,6000,Presidents Cup,Golf,88.33


In [17]:
to_remove = ['Matt Kuchar','Rickie Fowler','Hao-Tong Li','Cameron Smith']
df = df[~df.Name.isin(to_remove)]
df.tail()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
42,G,Abraham Ancer (13919913),Abraham Ancer,13919913,G,5000,Presidents Cup,Golf,66.21
43,G,Byeong Hun An (13919914),Byeong Hun An,13919914,G,4800,Presidents Cup,Golf,69.72
44,G,Joaquin Niemann (13919915),Joaquin Niemann,13919915,G,4600,Presidents Cup,Golf,71.17
46,G,Adam Hadwin (13919917),Adam Hadwin,13919917,G,4200,Presidents Cup,Golf,83.60
47,G,C.T. Pan (13919918),C.T. Pan,13919918,G,4000,Presidents Cup,Golf,51.43


In [11]:
lock = ['Adam Scott','Hideki Matsuyama','Louis Oosthuizen']

df = df[~df.Name.isin(lock)]
df[df['Roster Position']=='CPT']

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Justin Thomas (13919919),Justin Thomas,13919919,CPT,16200,Presidents Cup,Golf,102.70
1,G,Xander Schauffele (13919920),Xander Schauffele,13919920,CPT,15300,Presidents Cup,Golf,90.25
3,G,Dustin Johnson (13919922),Dustin Johnson,13919922,CPT,14100,Presidents Cup,Golf,105.75
6,G,Tiger Woods (13919925),Tiger Woods,13919925,CPT,12600,Presidents Cup,Golf,122.00
7,G,Patrick Cantlay (13919926),Patrick Cantlay,13919926,CPT,12300,Presidents Cup,Golf,95.67
8,G,Patrick Reed (13919927),Patrick Reed,13919927,CPT,11700,Presidents Cup,Golf,83.60
9,G,Webb Simpson (13919928),Webb Simpson,13919928,CPT,11100,Presidents Cup,Golf,107.33
13,G,Gary Woodland (13919930),Gary Woodland,13919930,CPT,9900,Presidents Cup,Golf,96.12
16,G,Marc Leishman (13919931),Marc Leishman,13919931,CPT,9300,Presidents Cup,Golf,66.80
20,G,Bryson DeChambeau (13919933),Bryson DeChambeau,13919933,CPT,8700,Presidents Cup,Golf,96.50


In [12]:
cpt_list = ['Patrick Cantlay','Patrick Reed','Webb Simpson','Gary Woodland','Bryson DeChambeau','Tony Finau']

In [14]:
df['Exposure'] = 0

## Generate lineups!

In [13]:
#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6'])
df_lineups

,G1,G2,G3,G4,G5,G6


680

In [62]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 85
min_sal = 49400
max_sal = 50000

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = []
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        #prit('picked player {}'.format(p))
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
        
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    total_own = df_now.Ownership.sum()
    total_sal = df_now.Salary.sum() + sal_used
    ##print(total_sal)
    if (total_sal < min_sal or total_sal > max_sal or total_own > 80):
        continue
        
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    curr_team = lock_full_names + df_now['Name + ID'].tolist()
    
    #Check if team has shown up before in df_lineups
    df3 = (df_lineups == np.array(curr_team))
    old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally increment m
    m += 1
    #print('m = {}'.format(m))

    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Name + ID','Actual Exposure','Exposure','Salary']]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Name + ID,Actual Exposure,Exposure,Salary
0,Viktor Hovland (13745197),20,46,11200
1,Jason Day (13745198),19,48,11000
2,Tony Finau (13745199),21,46,10600
3,Billy Horschel (13745200),25,50,10200
4,Joaquin Niemann (13745201),18,44,9800
5,Charles Howell III (13745202),23,44,9600
6,Aaron Wise (13745203),25,50,9400
7,Scottie Scheffler (13745206),20,44,9100
8,Denny McCarthy (13745213),17,44,8400
9,J.T. Poston (13745215),17,48,8200


In [63]:
for c in df_lineups.columns:
    df_lineups.rename(columns={c:'G'},inplace=True)

In [64]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/DK Entires.csv',index=False)

In [28]:
df_lineups.head()

,G,G,G,G,G,G
0,Matt Kuchar (13745196),Jason Day (13745198),Adam Schenk (13745233),Nick Taylor (13745243),Tom Hoge (13745277),Andrew Landry (13745288)
1,Tony Finau (13745199),Billy Horschel (13745200),Rory Sabbatini (13745221),Brian Harman (13745224),Kevin Chappell (13745262),Tom Hoge (13745277)
2,Jason Day (13745198),J.T. Poston (13745215),Dylan Frittelli (13745217),Rory Sabbatini (13745221),Carlos Ortiz (13745225),Nick Watney (13745251)
3,Jason Day (13745198),Billy Horschel (13745200),Bronson Burgoon (13745235),Adam Schenk (13745233),Nick Watney (13745251),Tom Hoge (13745277)
4,Jason Day (13745198),Billy Horschel (13745200),Sebastian Munoz (13745223),Kevin Streelman (13745231),Nick Watney (13745251),Andrew Landry (13745288)


# FD

In [29]:
df2 = pd.read_csv('Data/DailyRotoPGAProjections_Shriners Hospital for Children Open FD.csv')
#del df['Salary']
df2.head()

,Player,SlatePlayerID,Ownership,Sample,%:,Cut,Top 20,Top 5,Win,DK:,...,Scoring Pts,Points,Value,FD:,SD.1,Salary.1,Finish Pts.1,Scoring Pts.1,Points.1,Value.1
0,Brooks Koepka,38974-78367,16.54,150,NaN,0.810825,0.511350,0.214650,0.063875,NaN,...,83.711385,91.11,11.44,NaN,37.787783,12200.0,5.305312,88.744152,94.05,12.77
1,Patrick Cantlay,38974-79435,22.64,150,NaN,0.823925,0.522425,0.213650,0.060775,NaN,...,83.614958,91.05,12.53,NaN,36.715761,11800.0,5.375305,89.097241,94.47,14.19
2,Adam Scott,38974-78780,18.98,150,NaN,0.797225,0.471825,0.185025,0.050925,NaN,...,81.334255,88.07,11.53,NaN,37.297350,11400.0,4.880586,86.375940,91.26,13.03
3,Webb Simpson,38974-78804,13.54,150,NaN,0.786125,0.445225,0.165175,0.043200,NaN,...,79.074530,85.08,9.68,NaN,37.045458,11300.0,4.364660,83.958062,88.32,11.22
4,Hideki Matsuyama,38974-78508,17.87,150,NaN,0.765225,0.421750,0.153825,0.040150,NaN,...,78.440117,84.09,9.54,NaN,37.204530,11100.0,4.101694,83.068275,87.17,10.97


In [30]:
df2.columns

Index(['Player', 'SlatePlayerID', 'Ownership', 'Sample', '%:', 'Cut', 'Top 20',
       'Top 5', 'Win', 'DK:', 'SD', 'Salary', 'Finish Pts', 'Scoring Pts',
       'Points', 'Value', 'FD:', 'SD.1', 'Salary.1', 'Finish Pts.1',
       'Scoring Pts.1', 'Points.1', 'Value.1'],
      dtype='object')

In [31]:
#DF salary is in Salary.1 column
del df2['Salary']
df2.rename(columns={'Salary.1':'Salary'},inplace=True)
df2.sort_values(by='Salary',ascending=False).head()

,Player,SlatePlayerID,Ownership,Sample,%:,Cut,Top 20,Top 5,Win,DK:,...,Scoring Pts,Points,Value,FD:,SD.1,Salary,Finish Pts.1,Scoring Pts.1,Points.1,Value.1
0,Brooks Koepka,38974-78367,16.54,150,NaN,0.810825,0.511350,0.214650,0.063875,NaN,...,83.711385,91.11,11.44,NaN,37.787783,12200.0,5.305312,88.744152,94.05,12.77
1,Patrick Cantlay,38974-79435,22.64,150,NaN,0.823925,0.522425,0.213650,0.060775,NaN,...,83.614958,91.05,12.53,NaN,36.715761,11800.0,5.375305,89.097241,94.47,14.19
8,Bryson DeChambeau,38974-78005,12.43,150,NaN,0.722925,0.337950,0.108250,0.024250,NaN,...,74.359083,79.14,4.61,NaN,37.008679,11600.0,3.447313,78.506735,81.95,5.85
2,Adam Scott,38974-78780,18.98,150,NaN,0.797225,0.471825,0.185025,0.050925,NaN,...,81.334255,88.07,11.53,NaN,37.297350,11400.0,4.880586,86.375940,91.26,13.03
3,Webb Simpson,38974-78804,13.54,150,NaN,0.786125,0.445225,0.165175,0.043200,NaN,...,79.074530,85.08,9.68,NaN,37.045458,11300.0,4.364660,83.958062,88.32,11.22


In [32]:
#Add field with the player:SlatePlayerID that will go into lineup csvs
df2['Name + ID'] =  df2.SlatePlayerID  + ":" + df2.Player

In [33]:
#specify team settings

lock = ['Bryson DeChambeau','Viktor Hovland']
lock = ['Marc Leishman','Bubba Watson']
lock = ['Patrick Cantlay']
lock = []

to_remove = ['Charley Hoffman','Sam Ryder','Zach Johnson',
            'Jason Day','Sergio Garcia','Tiger Woods','Rafa Cabrera Bello',
            'Scott Piercy']
to_remove = ['Kevin Chappell','Matt Jones']

max_sal = 910200

In [34]:
#Now pick a penalty, and find 1st, 2nd, 3rd, ... opt teams
#df = df_all.copy()
df_now = df2.copy()

lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df_now[df_now.Player==p]['Name + ID'].iloc[0])
sal_used = df_now[df_now.Player.isin(lock)].Salary.sum()
sal_left = 60000-sal_used
players_left = 6-len(lock)
print(lock_full_names)

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Player.isin(lock)]
df_now = df_now[~df_now.Player.isin(to_remove)]

penalty = 0.3
opt_teams = []
for i in np.arange(5): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = sal_left
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap]

    # Form objective.
    obj = cp.Maximize(x*(p-penalty*own))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('Team {} (obj = {:.2f}): '.format(i+1,prob.value),opt_team)
    opt_teams.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Player.isin(opt_team)]


[]
Team 1 (obj = 445.71):  ['Brooks Koepka', 'Adam Scott', 'Jason Kokrak', 'Abraham Ancer', 'Jim Furyk', 'Sepp Straka']
Team 2 (obj = 438.63):  ['Patrick Cantlay', 'Webb Simpson', 'Hideki Matsuyama', 'Kevin Streelman', 'Kyle Stanley', 'Harry Higgs']
Team 3 (obj = 430.55):  ['Tony Finau', 'Collin Morikawa', 'Chez Reavie', 'Byeong Hun An', 'Emiliano Grillo', 'Keith Mitchell']
Team 4 (obj = 419.23):  ['Bryson DeChambeau', 'Brandt Snedeker', 'Andrew Putnam', 'Brian Harman', 'Sebastian Munoz', 'Ryan Moore']
Team 5 (obj = 413.56):  ['Joaquin Niemann', 'Adam Hadwin', 'Charles Howell III', 'Dylan Frittelli', 'Rory Sabbatini', 'Lucas Glover']


In [35]:
#Find opt teams with max own cap

df_now = df2.copy()

lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df_now[df_now.Player==p]['SlatePlayerID'].iloc[0]+":"+p)
sal_used = df_now[df_now.Player.isin(lock)].Salary.sum()
own_used = df_now[df_now.Player.isin(lock)].Ownership.sum()
sal_left = 60000-sal_used
players_left = 6-len(lock)
print(lock_full_names)

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Player.isin(lock)]
df_now = df_now[~df_now.Player.isin(to_remove)]


own_cap = 100 - own_used
#print(own_cap)
opt_teams_cap = []
for i in np.arange(11): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = sal_left
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap,
                  x*own <= own_cap]

    # Form objective.
    obj = cp.Maximize(x*(p))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('Team {} (obj = {:.2f}): '.format(i+1,prob.value),opt_team)
    opt_teams_cap.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Player.isin(opt_team)]

[]
Team 1 (obj = 467.90):  ['Brooks Koepka', 'Patrick Cantlay', 'Adam Scott', 'Jason Kokrak', 'Sepp Straka', 'Ryan Brehm']
Team 2 (obj = 459.58):  ['Webb Simpson', 'Hideki Matsuyama', 'Collin Morikawa', 'Jim Furyk', 'Rory Sabbatini', 'Kyle Stanley']
Team 3 (obj = 449.80):  ['Tony Finau', 'Chez Reavie', 'Byeong Hun An', 'Abraham Ancer', 'Aaron Wise', 'Brian Harman']
Team 4 (obj = 436.96):  ['Gary Woodland', 'Joaquin Niemann', 'Adam Hadwin', 'Andrew Putnam', 'Kevin Streelman', 'Sebastian Munoz']
Team 5 (obj = 429.71):  ['Bryson DeChambeau', 'Brandt Snedeker', 'Charles Howell III', 'Dylan Frittelli', 'Emiliano Grillo', 'Keith Mitchell']
Team 6 (obj = 412.82):  ['Scottie Scheffler', 'J.T. Poston', 'Ryan Moore', 'Lucas Glover', 'Scott Piercy', 'Bubba Watson']
Team 7 (obj = 396.83):  ['Keegan Bradley', 'Harold Varner III', 'Matthew Wolff', 'Russell Knox', 'Cameron Smith', 'Zach Johnson']
Team 8 (obj = 384.47):  ['Jhonattan Vegas', 'Ryan Palmer', 'Bud Cauley', 'Bronson Burgoon', 'Russell Henl

In [36]:
### Assign exposures for top j teams
exposures = [50,45]
if len(exposures)<len(opt_teams):
    exposures = exposures + [0]*(len(opt_teams)-len(exposures))
exposures

[50, 45, 0, 0, 0]

In [37]:
#Create df to hold player pool
df_teams = df2[['Name + ID','Player','Salary','Ownership']]
df_teams.sort_values(by='Salary',ascending=False,inplace=True)
df_teams['Exposure'] = 0
df_teams['Actual Exposure'] = 0
df_teams.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

,Name + ID,Player,Salary,Ownership,Exposure,Actual Exposure
0,38974-78367:Brooks Koepka,Brooks Koepka,12200.0,16.54,0,0
1,38974-79435:Patrick Cantlay,Patrick Cantlay,11800.0,22.64,0,0
8,38974-78005:Bryson DeChambeau,Bryson DeChambeau,11600.0,12.43,0,0
2,38974-78780:Adam Scott,Adam Scott,11400.0,18.98,0,0
3,38974-78804:Webb Simpson,Webb Simpson,11300.0,13.54,0,0


In [38]:
#Add in exposures for players in player pool
for i, team in enumerate(opt_teams):
    df_teams.Exposure = np.where(df_teams.Player.isin(team),exposures[i],df_teams.Exposure)
    
#OPTIONAL: Add or adjust players
#more_team = {'Francesco Molinari':20,'Henrik Stenson':20,'Lucas Glover':20}
more_team = {'Xinjun Zhang':35}
#more_team = {}
for p in more_team.keys():
    df_teams.Exposure = np.where(df_teams.Player==p,more_team[p],df_teams.Exposure)
    
df_teams = df_teams[df_teams.Exposure > 0].reset_index(drop=True)
df_teams

,Name + ID,Player,Salary,Ownership,Exposure,Actual Exposure
0,38974-78367:Brooks Koepka,Brooks Koepka,12200.0,16.54,50,0
1,38974-79435:Patrick Cantlay,Patrick Cantlay,11800.0,22.64,45,0
2,38974-78780:Adam Scott,Adam Scott,11400.0,18.98,50,0
3,38974-78804:Webb Simpson,Webb Simpson,11300.0,13.54,45,0
4,38974-78508:Hideki Matsuyama,Hideki Matsuyama,11100.0,17.87,45,0
5,38974-78372:Jason Kokrak,Jason Kokrak,10000.0,13.43,50,0
6,38974-77806:Abraham Ancer,Abraham Ancer,9600.0,5.88,50,0
7,38974-78106:Jim Furyk,Jim Furyk,9200.0,10.88,50,0
8,38974-78857:Kevin Streelman,Kevin Streelman,9100.0,6.55,45,0
9,38974-78841:Kyle Stanley,Kyle Stanley,8500.0,6.88,45,0


In [39]:
#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6'])
df_lineups

,G1,G2,G3,G4,G5,G6


In [40]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 32
min_sal = 58500
max_sal = 59800

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = []
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        #prit('picked player {}'.format(p))
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
        
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    total_own = df_now.Ownership.sum()
    total_sal = df_now.Salary.sum() + sal_used
    ##print(total_sal)
    if (total_sal < min_sal or total_sal > max_sal or total_own > 85):
        continue
        
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    curr_team = lock_full_names + df_now['Name + ID'].tolist()
    
    #Check if team has shown up before in df_lineups
    df3 = (df_lineups == np.array(curr_team))
    old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally increment m
    m += 1
    #print('m = {}'.format(m))
    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Player','Name + ID','Actual Exposure']]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Player,Name + ID,Actual Exposure
0,Brooks Koepka,38974-78367:Brooks Koepka,13
1,Patrick Cantlay,38974-79435:Patrick Cantlay,14
2,Adam Scott,38974-78780:Adam Scott,17
3,Webb Simpson,38974-78804:Webb Simpson,18
4,Hideki Matsuyama,38974-78508:Hideki Matsuyama,14
5,Jason Kokrak,38974-78372:Jason Kokrak,14
6,Abraham Ancer,38974-77806:Abraham Ancer,17
7,Jim Furyk,38974-78106:Jim Furyk,18
8,Kevin Streelman,38974-78857:Kevin Streelman,13
9,Kyle Stanley,38974-78841:Kyle Stanley,16


In [41]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/FD Entires.csv')

In [42]:
df_lineups

,G1,G2,G3,G4,G5,G6
0,38974-78367:Brooks Koepka,38974-78780:Adam Scott,38974-78804:Webb Simpson,38974-78857:Kevin Streelman,38974-80579:Sepp Straka,38974-79022:Xinjun Zhang
1,38974-79435:Patrick Cantlay,38974-78508:Hideki Matsuyama,38974-78372:Jason Kokrak,38974-77806:Abraham Ancer,38974-78841:Kyle Stanley,38974-116964:Harry Higgs
2,38974-78367:Brooks Koepka,38974-79435:Patrick Cantlay,38974-77806:Abraham Ancer,38974-78857:Kevin Streelman,38974-78841:Kyle Stanley,38974-79022:Xinjun Zhang
3,38974-78367:Brooks Koepka,38974-78508:Hideki Matsuyama,38974-77806:Abraham Ancer,38974-78106:Jim Furyk,38974-78857:Kevin Streelman,38974-78841:Kyle Stanley
4,38974-79435:Patrick Cantlay,38974-78804:Webb Simpson,38974-78372:Jason Kokrak,38974-78106:Jim Furyk,38974-78857:Kevin Streelman,38974-80579:Sepp Straka
5,38974-78780:Adam Scott,38974-78804:Webb Simpson,38974-78372:Jason Kokrak,38974-77806:Abraham Ancer,38974-78857:Kevin Streelman,38974-79022:Xinjun Zhang
6,38974-78367:Brooks Koepka,38974-79435:Patrick Cantlay,38974-78804:Webb Simpson,38974-78841:Kyle Stanley,38974-116964:Harry Higgs,38974-80579:Sepp Straka
7,38974-78780:Adam Scott,38974-78804:Webb Simpson,38974-78372:Jason Kokrak,38974-77806:Abraham Ancer,38974-78841:Kyle Stanley,38974-116964:Harry Higgs
8,38974-78780:Adam Scott,38974-78804:Webb Simpson,38974-78372:Jason Kokrak,38974-77806:Abraham Ancer,38974-78857:Kevin Streelman,38974-116964:Harry Higgs
9,38974-78780:Adam Scott,38974-78804:Webb Simpson,38974-78508:Hideki Matsuyama,38974-78106:Jim Furyk,38974-78857:Kevin Streelman,38974-79022:Xinjun Zhang
